In [3]:
from dotenv import load_dotenv
load_dotenv('/Users/eshantdas/Desktop/Youtube/Youtube_GenAI_Test_code/myenv/.env')

True

In [4]:

from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

model=ChatGroq( model="gemma-7b-it")


In [5]:
from langchain_core.pydantic_v1 import BaseModel, Field


In [6]:
class Tagging(BaseModel):
    """Put the sentiment and language in the form of a dictionary"""
    sentiment:str=Field(description="sentiment of the text which should be 'pos','neg' or 'neutral")
    language:str=Field(description="language of the text")

In [7]:
prompt=ChatPromptTemplate.from_messages([
    ("system","Think carefully and then tag the text as instrucred"),
    ("user","{input}")
]
)

In [8]:
chain=(prompt | model.bind_tools(
    tools=[Tagging],tool_choice='Tagging').with_config(run_name='Tagging')
    | PydanticToolsParser(tools=[Tagging])
    )

chain.invoke({"input":"I love Eshant"})

[Tagging(sentiment='positive', language='en')]

In [9]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

from langchain_core.utils.function_calling import convert_to_openai_function

In [10]:
convert_to_openai_function(Tagging)

{'name': 'Tagging',
 'description': 'Put the sentiment and language in the form of a dictionary',
 'parameters': {'type': 'object',
  'properties': {'sentiment': {'description': "sentiment of the text which should be 'pos','neg' or 'neutral",
    'type': 'string'},
   'language': {'description': 'language of the text', 'type': 'string'}},
  'required': ['sentiment', 'language']}}

In [11]:
import json
def output_parser_letsgo(data):
    a=data
    b=a.additional_kwargs['function_call']['arguments']
    c=json.loads(b)
    return c
    
    

In [12]:
import json
def output_parser_letsgo(data):
    a=data
    b=a.additional_kwargs['function_call']['arguments']
    c=json.loads(b)
    return c
    
    


from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableLambda
parser=JsonOutputParser(pydantic_object=Tagging)

tagging_function=[convert_to_openai_function(Tagging)]
model_with_function=model.bind(
    functions=tagging_function,
    function_call={"name":"Tagging"}
)

tagging_chain=prompt | model_with_function | RunnableLambda(output_parser_letsgo)

In [13]:
a=tagging_chain.invoke({"input":"i love langchain"})
a

{'language': 'english', 'sentiment': 'pos'}

## `Extraction`

In [14]:
from typing import Optional,List
class Person(BaseModel):
    """Information about a person"""
    name:str=Field(description="persons name")
    age:Optional[int]=Field(description="person's age")
    hobby:str=Field(description="person hobby")

In [15]:
class Information(BaseModel):
    """Information to extract"""
    person:List[Person]=Field(description="List of info about people")

In [16]:
extraction_function=[convert_to_openai_function(Information)]
extraction_model=model.bind(
    functions=extraction_function,
    function_call={"name":"Information"}
)

In [17]:
extraction_chain=prompt | extraction_model

extraction_chain.invoke("Eshant is 20 years old and he likes to play Cricketm his mom loves to cook Bengali food")

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n"tool_call": {\n"id": "pending",\n"type": "function",\n"function": {\n"name": "Information"\n},\n"parameters": {\n"properties": {\n"person": {\n"items": {\n"properties": {\n"age": {"type": "integer"},\n"hobby": {"type": "string"},\n"name": {"type": "string"}\n},\n"required": ["name", "hobby"],\n"type": "array"\n}\n}\n}\n}\n</tool-use>'}}